In [ ]:
!pip install tensorflow opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define paths (use Google Drive for storage)
# from google.colab import drive
# drive.mount('/content/drive')


original_images_path = '/content/drive/MyDrive/pixelation/original_images'
pixelated_images_path = '/content/drive/MyDrive/pixelation/pixelated_images'
dataset_path = '/content/drive/MyDrive/pixelation/dataset'

# Create pixelated images
def create_pixelated_images():
    if not os.path.exists(pixelated_images_path):
        os.makedirs(pixelated_images_path)

    for filename in os.listdir(original_images_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(original_images_path, filename))
            if img is None:
                continue

            # JPEG compression
            for quality in [10, 20]:
                compressed_img_path = os.path.join(pixelated_images_path, f'{filename}_quality_{quality}.jpg')
                cv2.imwrite(compressed_img_path, img, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

            # Downscale-upscale
            for scale in [5, 6]:
                downscaled_img = cv2.resize(img, (img.shape[1]//scale, img.shape[0]//scale), interpolation=cv2.INTER_NEAREST)
                upscaled_img = cv2.resize(downscaled_img, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
                upscaled_img_path = os.path.join(pixelated_images_path, f'{filename}_scale_{scale}.jpg')
                cv2.imwrite(upscaled_img_path, upscaled_img)

create_pixelated_images()

# Prepare dataset
def prepare_dataset():
    images = []
    labels = []

    # Load original images
    for filename in tqdm(os.listdir(original_images_path)):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = cv2.imread(os.path.join(original_images_path, filename))
            if img is not None:
                images.append(cv2.resize(img, (224, 224)))
                labels.append(0)  # Label 0 for original images

    # Load pixelated images
    for filename in tqdm(os.listdir(pixelated_images_path)):
        if filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(pixelated_images_path, filename))
            if img is not None:
                images.append(cv2.resize(img, (224, 224)))
                labels.append(1)  # Label 1 for pixelated images

    images = np.array(images)
    labels = np.array(labels)

    # Split the dataset
    X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    np.save(os.path.join(dataset_path, 'X_train.npy'), X_train)
    np.save(os.path.join(dataset_path, 'y_train.npy'), y_train)
    np.save(os.path.join(dataset_path, 'X_val.npy'), X_val)
    np.save(os.path.join(dataset_path, 'y_val.npy'), y_val)
    np.save(os.path.join(dataset_path, 'X_test.npy'), X_test)
    np.save(os.path.join(dataset_path, 'y_test.npy'), y_test)

prepare_dataset()

100%|██████████| 860/860 [00:38<00:00, 22.59it/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load dataset
dataset_path = '/content/drive/MyDrive/pixelation/dataset'
X_train = np.load(os.path.join(dataset_path, 'X_train.npy'))
y_train = np.load(os.path.join(dataset_path, 'y_train.npy'))
X_val = np.load(os.path.join(dataset_path, 'X_val.npy'))
y_val = np.load(os.path.join(dataset_path, 'y_val.npy'))
X_test = np.load(os.path.join(dataset_path, 'X_test.npy'))
y_test = np.load(os.path.join(dataset_path, 'y_test.npy'))

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Build the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10

model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=(X_val, y_val),
    epochs=epochs
)

# Unfreeze the base model for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model
model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=(X_val, y_val),
    epochs=10
)

# Save the model
model_path = '/content/drive/MyDrive/pixelation/pixelation_detection_model.h5'
model.save(model_path)


9406464/9406464 [==============================] - 0s 0us/step


Epoch 1/10
26/26 [==============================] - 49s 2s/step - loss: 0.6286 - accuracy: 0.7222 - val_loss: 0.5509 - val_accuracy: 0.7850
Epoch 2/10
26/26 [==============================] - 47s 2s/step - loss: 0.5405 - accuracy: 0.7983 - val_loss: 0.5327 - val_accuracy: 0.7850
Epoch 3/10
26/26 [==============================] - 47s 2s/step - loss: 0.5129 - accuracy: 0.7911 - val_loss: 0.5415 - val_accuracy: 0.7850
Epoch 4/10
26/26 [==============================] - 44s 2s/step - loss: 0.4867 - accuracy: 0.7995 - val_loss: 0.5517 - val_accuracy: 0.7850
Epoch 5/10
26/26 [==============================] - 47s 2s/step - loss: 0.4770 - accuracy: 0.8031 - val_loss: 0.5506 - val_accuracy: 0.7850
Epoch 6/10
26/26 [==============================] - 48s 2s/step - loss: 0.5008 - accuracy: 0.7899 - val_loss: 0.5385 - val_accuracy: 0.7850
Epoch 7/10
26/26 [==============================] - 48s 2s/step - loss: 0.4848 - accuracy: 0.7983 - val_loss: 0.5306 - val_accuracy: 0.7850
Epoch 8/10
26/26 [==

Epoch 1/10
26/26 [==============================] - 174s 6s/step - loss: 0.5349 - accuracy: 0.7874 - val_loss: 0.5798 - val_accuracy: 0.7850
Epoch 2/10
26/26 [==============================] - 154s 6s/step - loss: 0.4388 - accuracy: 0.7959 - val_loss: 0.5290 - val_accuracy: 0.7850
Epoch 3/10
26/26 [==============================] - 147s 6s/step - loss: 0.4376 - accuracy: 0.7983 - val_loss: 0.6414 - val_accuracy: 0.7196
Epoch 4/10
26/26 [==============================] - 141s 5s/step - loss: 0.3948 - accuracy: 0.8345 - val_loss: 1.6115 - val_accuracy: 0.2243
Epoch 5/10
26/26 [==============================] - 146s 6s/step - loss: 0.3965 - accuracy: 0.8321 - val_loss: 0.4772 - val_accuracy: 0.7570
Epoch 6/10
26/26 [==============================] - 147s 6s/step - loss: 0.3527 - accuracy: 0.8522 - val_loss: 1.4010 - val_accuracy: 0.2897
Epoch 7/10
26/26 [==============================] - 145s 6s/step - loss: 0.3524 - accuracy: 0.8394 - val_loss: 0.9299 - val_accuracy: 0.2710
Epoch 8/10
26

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Load the trained model
model_path = '/content/drive/MyDrive/pixelation/pixelation_detection_model.h5'
model = tf.keras.models.load_model(model_path)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the optimized model
tflite_model_path = '/content/drive/MyDrive/pixelation/pixelation_detection_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# Example code for running inference on a TFLite model
def run_inference(image_path, model_path):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Prepare the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0).astype(np.float32)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()

    # Get the result
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = output_data[0]

    return 'Pixelated' if prediction > 0.5 else 'Non-Pixelated'

# Test the inference function
test_image_path = '/content/drive/MyDrive/pixelation/pixelated_images/cherry_37.jpg_quality_10.jpg'
print(run_inference(test_image_path, tflite_model_path))


Pixelated


In [ ]:
test_image_path = '/content/drive/MyDrive/pixelation/pixelated_images/cherry_37.jpg_scale_5.jpg'
print(run_inference(test_image_path, tflite_model_path))

Non-Pixelated


In [ ]:
if os.path.exists(tflite_model_path):
    model_size = os.path.getsize(tflite_model_path)
    print(f"Model size: {model_size / (1024 * 1024):.2f} MB")  # Convert to MB
else:
    print("Model file not found.")

Model size: 9.07 MB


In [ ]:
model = tf.keras.applications.MobileNetV2()  # Example model

# Save the model to a temporary file
model.save('/tmp/model')

# Calculate the size of the saved model file
model_size = os.path.getsize('/tmp/model')
print(f"Model size: {model_size / (1024 * 1024):.2f} MB")  # Convert to MB


14536120/14536120 [==============================] - 0s 0us/step


Model size: 0.00 MB


In [ ]:
import time

# Assume `model` is already loaded and ready for inference

# Example inference timing
start_time = time.time()
test_image_path = '/content/drive/MyDrive/pixelation/pixelated_images/cherry_37.jpg_quality_10.jpg'
print(run_inference(test_image_path, tflite_model_path))
end_time = time.time()

inference_time = end_time - start_time
fps = 1 / inference_time

print(f"Inference Time: {inference_time:.4f} seconds")
print(f"FPS: {fps:.2f}")


Pixelated
Inference Time: 0.2169 seconds
FPS: 4.61
